# Skin Cancer detection

In [1]:
import keras
import numpy as np
import pandas as pd
from sklearn.datasets import load_files       
from keras.utils import np_utils
from glob import glob
from scipy import stats
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
# import cv2



Using TensorFlow backend.


In [2]:
train_path = "https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/skin-cancer/train.zip"
test_path = "https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/skin-cancer/test.zip"
validation_path = "https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/skin-cancer/valid.zip"

In [3]:
def download_zip_and_extract(remote_path, local_path):

    import requests, zipfile, io
    r = requests.get(remote_path)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(local_path)

In [4]:
#download_zip_and_extract(validation_path, "data")

In [5]:
#download_zip_and_extract(train_path, "data")

In [6]:
#download_zip_and_extract(test_path, "data")

In [7]:
# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    files = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']), 3)
    return files, targets

In [8]:
train_files, train_targets = load_dataset('data/train')

In [9]:
test_files, test_targets = load_dataset('data/test')

In [10]:
valid_files, valid_targets = load_dataset('data/valid')

In [11]:
valid_files.size

150

In [12]:
#extracting the names from the folder names in sorted order
labels = sorted([label[11:-1] for label in glob('data/train/*/')])

In [13]:
# print statistics about the dataset
print('There are %d total lesion categories.' % len(labels))
print('There are %s total lesion images.\n' % str(len(train_files) + len(valid_files) + len(test_files)))
print('There are %d training lesion images.' % len(train_files))
print('There are %d validation lesion images.' % len(valid_files))
print('There are %d test lesion images.'% len(test_files))

There are 3 total lesion categories.
There are 2750 total lesion images.

There are 2000 training lesion images.
There are 150 validation lesion images.
There are 600 test lesion images.


## Visualising sample images

In [14]:
# def visualize_img(img_path, ax):
#     import matplotlib.pyplot as plt
#     %matplotlib inline

#     img = cv2.imread(img_path)
#     ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [15]:
# fig = plt.figure(figsize=(20, 10))
# for i in range(12):
#     ax = fig.add_subplot(3, 4, i + 1, xticks=[], yticks=[])
#     visualize_img(train_files[i], ax)

Creating a pretrained model

In [16]:
# sample = cv2.imread(train_files[0])

# sample.shape


In [17]:
model = keras.applications.VGG19(
    include_top=False, 
    weights='imagenet', 
    input_tensor=None, 
    input_shape=(254, 254, 3), 
    pooling=None)


In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 254, 254, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 254, 254, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 254, 254, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 127, 127, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 127, 127, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 127, 127, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 63, 63, 128)       0         
__________

In [19]:
len(model.layers)

22

## Freeze layers

In [20]:
# Freeze the layers which you don't want to train. Here I am freezing the first 5 layers.
for layer in model.layers[:22]:
    layer.trainable = False

Adding custom layers

In [21]:
#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
predictions = Dense(3, activation="softmax")(x)

In [22]:
# creating the final model 
model_final = Model(input = model.input, output = predictions)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  from ipykernel import kernelapp as app


In [23]:
# compile the model 
model_final.compile(
    loss = "categorical_crossentropy", 
    optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), 
    metrics=["accuracy"])


In [24]:
train_data_dir = 'data/train'
validation_data_dir = 'data/valid'
batch_size = 100

# Initiate the train and test generators with data Augumentation 
train_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.1,
width_shift_range = 0.1,
height_shift_range=0.1,
rotation_range=10)

test_datagen = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = True,
fill_mode = "nearest",
zoom_range = 0.1,
width_shift_range = 0.1,
height_shift_range=0.1,
rotation_range=10)

train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size = (254, 254),
batch_size = batch_size, 
class_mode = "categorical")

validation_generator = test_datagen.flow_from_directory(
validation_data_dir,
target_size = (254,254),
batch_size = batch_size, 
class_mode = "categorical")

Found 2000 images belonging to 3 classes.
Found 150 images belonging to 3 classes.


In [25]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("vgg16_1-best-classifier.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')


In [26]:
steps_per_epoch = 10
epochs = 10

In [ ]:
# Train the model 
model_final.fit_generator(
train_generator,
steps_per_epoch = steps_per_epoch,
epochs = epochs,
validation_data = validation_generator,
validation_steps = valid_files.size / batch_size,
callbacks = [checkpoint, early])

Epoch 1/10
10/10 [==============================] - 188s 19s/step - loss: 1.2248 - acc: 0.4680 - val_loss: 1.7142 - val_acc: 0.5200

Epoch 00001: val_acc improved from -inf to 0.52000, saving model to vgg16_1-best-classifier.h5
Epoch 2/10
10/10 [==============================] - 171s 17s/step - loss: 1.0708 - acc: 0.6720 - val_loss: 1.4666 - val_acc: 0.5200

Epoch 00002: val_acc did not improve from 0.52000
Epoch 3/10
10/10 [==============================] - 175s 18s/step - loss: 0.9994 - acc: 0.6000 - val_loss: 1.2001 - val_acc: 0.5200

Epoch 00003: val_acc did not improve from 0.52000
Epoch 4/10
10/10 [==============================] - 169s 17s/step - loss: 0.9660 - acc: 0.6380 - val_loss: 1.1985 - val_acc: 0.5200

Epoch 00004: val_acc did not improve from 0.52000
Epoch 5/10
10/10 [==============================] - 165s 16s/step - loss: 0.9622 - acc: 0.6190 - val_loss: 1.1436 - val_acc: 0.5200

Epoch 00005: val_acc improved from 0.52000 to 0.52000, saving model to vgg16_1-best-classi

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())